# FINETUNING - Model Training

Fine-tuning is a process of adapting a pre-trained model to a specific task by training it on a smaller, task-specific dataset. Fine-tuning GPT for text generation involves adjusting the model’s weights so it can produce relevant and coherent text outputs for a given prompt.

is notebook documents the process of **fine-tuning a GPT model** to specialize in answering questions about **Ikorodu scholarships**. The goal is to create an AI model that provides **accurate, relevant, and up-to-date** information on available scholarships, eligibility criteria, and application processes. 
 
## Fine-Tuning Process  
1. **Dataset Preparation** – Collected and curated scholarship-related data specific to Ikorodu.  
2. **Data Formatting** – Structured the data into OpenAI’s fine-tuning format (JSONL).  
3. **Model Training** – Used OpenAI’s fine-tuning API to adapt GPT to the scholarship domain.  
4. **Evaluation & Testing** – Assessed model performance using sample queries.  
5. **Optimization** – Adjusted hyperparameters, models and refined the dataset for improved accuracy.  

This notebook contains all the key steps taken to **fine-tune and evaluate** the model using OpenAI's finetuning API.  


# Install dependencies

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [9]:
from openai import OpenAI

In [10]:
client = OpenAI()

# TESTING THE BASE MODEL

In [12]:
OgaScholar = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "I'm from Abeokuta, can i apply for IKODASS scholarship?"},
    ]
)

OgaScholar.choices[0].message.content

'Yes, you can apply for the IKODASS scholarship regardless of your location, as long as you meet the eligibility criteria specified by the IKODASS scholarship program. Make sure to review the specific requirements and guidelines provided by IKODASS to determine if you qualify for the scholarship.'

## FINE TUNING

In [37]:
import openai
response = openai.files.create(
    file=open("ikorodu_scholarship.jsonl", "rb"),
    purpose="fine-tune"
)

print(response)

FileObject(id='file-2Z9jSYVFdxFvrF1BNsAfUS', bytes=8555, created_at=1741348502, filename='ikorodu_scholarship.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)


In [38]:
fine_tune_response = client.fine_tuning.jobs.create(
    training_file="file-2Z9jSYVFdxFvrF1BNsAfUS",
    model="gpt-4o-2024-08-06"
)

print(fine_tune_response)

FineTuningJob(id='ftjob-kqSTLdT7qZDLBXMwKc1boMiJ', created_at=1741348523, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-o9FsB3CxwUlYPqpAyqiGW1xI', result_files=[], seed=1243452803, status='validating_files', trained_tokens=None, training_file='file-2Z9jSYVFdxFvrF1BNsAfUS', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)


In [39]:
print(fine_tune_response)

FineTuningJob(id='ftjob-kqSTLdT7qZDLBXMwKc1boMiJ', created_at=1741348523, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-o9FsB3CxwUlYPqpAyqiGW1xI', result_files=[], seed=1243452803, status='validating_files', trained_tokens=None, training_file='file-2Z9jSYVFdxFvrF1BNsAfUS', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)


In [54]:
# List fine-tuning jobs
fine_tune_jobs = openai.fine_tuning.jobs.list(limit=10)

# Extract fine-tuned model names
for job in fine_tune_jobs.data:
    if job.fine_tuned_model:  # Ensure it's completed
        print("Fine-tuned model ID:", job.fine_tuned_model)


Fine-tuned model ID: ft:gpt-4o-2024-08-06:personal::B8QmIcnz
Fine-tuned model ID: ft:gpt-4o-2024-08-06:personal::B8QIDV0l
Fine-tuned model ID: ft:gpt-4o-2024-08-06:personal::B8QIuNqY
Fine-tuned model ID: ft:gpt-4o-2024-08-06:personal::B8QH5Zf1
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal::B8QBqURx
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal::B8PJKsGD
Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal::B8PIJWKL


## USING THE FINE TUNED MODEL

In [12]:
#creating function to get response
def get_oga_scholar_response(user_input):
    response = client.chat.completions.create(
        model = "ft:gpt-4o-2024-08-06:personal::B8QmIcnz",
        messages = [
            {"role": "system", "content": "You are an advisor for ikorodu students on scholarship available for ikorodu indigene  and resident only"},
            {"role": "user", "content": user_input},
        ]
    )
    return response.choices[0].message

In [26]:
user_input = "I'm from Abeokuta, can i apply for IKODASS scholarship??"
answer = get_oga_scholar_response(user_input)
print(answer.content)


The IKODASS Scholarship is specifically for Ikorodu indigenes and residents. You may not be eligible if you're from Abeokuta.


In [27]:
user_input = "Hello??"
answer = get_oga_scholar_response(user_input)
print(answer.content)

Hi there! How can I assist you with scholarships in Ikorodu?


In [29]:
user_input = "My cgpa is 3.0. I don't if there is any scholarship that i can apply for in Ikorodu? and can you list the available scholarships for Ikorodu students?"
answer = get_oga_scholar_response(user_input)
print(answer.content)

In Ikorodu, most scholarships have a minimum CGPA requirement of 3.0. Scholarships are for Ikorodu indigenes and residents and may require a LGA identification letter. Here are some scholarships you might find useful: 

### Ikorodu Division Peace Initiative Foundation (IDPIF) Scholarship
- **Eligibility**: Open to students in Ikorodu including full-time undergraduates with a minimum CGPA of 3.0.
  
### Ijede LCDA Scholarship
- **Eligibility**: For Indigenes of Ijede LCDA with at least a CGPA of 3.0.

### Oriwu Club Scholarship
- **Eligibility**: Open to Ikorodu indigenes, focusing on academic merit (usually CGPA of 4.0 and above) and financial need.

### Lagos State Scholarship Board (LSSB)
- **Eligibility**: Open to Lagos State indigenes, requires a minimum CGPA of 3.5. (Separate from Ikorodu-specific scholarships, but noteworthy).

These scholarships may include academic merit, financial need, and interviews. Each has specific requirements, so ensure you meet eligibility criteria bef

In [30]:
user_input = "do you know applisso?"
answer = get_oga_scholar_response(user_input)
print(answer.content)

Yes, Applisso offers scholarships to Ikorodu indigenes and residents. Would you like more details on their eligibility and application process?
